### Adding Geographical Coordinates

In [62]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pgeocode

from geopy.geocoders import Nominatim


In [20]:
df = pd.read_csv('../outputs/IPLDS_all_network_features.csv').sort_values(by=['hsa','year'])
df.head()

,hsa,year,assortativity,average_clustering,density,nnodes,nedges,forman_mean,forman_stdev,forman_median,...,forman_quantile_25,forman_quantile_50,forman_quantile_75,centrality_mean,centrality_stdev,centrality_median,centrality_mode,centrality_quantile_25,centrality_quantile_50,centrality_quantile_75
5686,1001,2014,-0.429306,0.757948,0.308016,98,1464,-2.997268,29.105001,-2.0,...,-23.0,-2.0,14.0,0.308016,0.240908,0.278351,0.010309,0.108247,0.278351,0.420103
10049,1001,2015,-0.412140,0.705853,0.277628,106,1545,-3.008414,29.637518,-4.0,...,-23.0,-4.0,12.0,0.277628,0.231627,0.252381,0.009524,0.073810,0.252381,0.369048
13095,1001,2016,-0.441431,0.721651,0.256711,109,1511,-11.625414,30.185857,-13.0,...,-33.0,-13.0,5.0,0.256711,0.223112,0.194444,0.018519,0.092593,0.194444,0.333333
5519,1001,2017,-0.389357,0.756683,0.242869,110,1456,-11.618819,27.342072,-10.0,...,-31.0,-10.0,4.0,0.242869,0.210172,0.192661,0.027523,0.064220,0.192661,0.330275
6158,1002,2014,-0.276127,0.853843,0.620120,37,413,13.883777,12.185454,16.0,...,8.0,16.0,23.0,0.620120,0.262655,0.694444,0.805556,0.500000,0.694444,0.805556


In [21]:
# Initialize pgeocode for the United States
nomi = pgeocode.Nominatim('us')

In [39]:
hsa_geo2017 = pd.read_csv("/Users/jeremy.wayland/Desktop/dev/apparent/outputs/databases/hosp17_atlas.csv")
hsa_geo2017.columns = hsa_geo2017.columns.str.lower()
hsa_geo2017['year'] = 2017
hsa_geo2017.head()

,provider,new_prov,status17,shortname,pname,pcity,pstate,pzip,hsa,hsaname,...,posbeds,sysname,ahabeds,category,teachstatus,hosptype,nci_center,excl_flag,icu_flag,year
0,10001,010001,9.OK?,Southeast Alabama Medical Center,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,1026,AL- DOTHAN,...,408,NaN,387.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2017
1,10005,010005,9.OK?,Marshall Medical Center South,MARSHALL MEDICAL CENTER SOUTH,BOAZ,AL,35957,1012,AL- BOAZ,...,240,Marshall Health System,114.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2017
2,10006,010006,9.OK?,Eliza Coffee Memorial Hospital,ELIZA COFFEE MEMORIAL HOSPITAL,FLORENCE,AL,35631,1036,AL- FLORENCE,...,358,RCCH HealthCare Partners,278.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2017
3,10007,010007,9.OK?,Mizell Memorial Hospital,MIZELL MEMORIAL HOSPITAL,OPP,AL,36467,1065,AL- OPP,...,85,NaN,59.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2017
4,10008,010008,9.OK?,Crenshaw Community Hospital,CRENSHAW COMMUNITY HOSPITAL,LUVERNE,AL,36049,1056,AL- LUVERNE,...,45,NaN,65.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,noICUdays,2017


In [38]:
hsa_geo2016 = pd.read_csv("~/Downloads/hosp16_atlas.csv")
hsa_geo2016.columns = hsa_geo2016.columns.str.lower()
hsa_geo2015['year'] = 2016
hsa_geo2016.head()

,provider,new_prov,status16,shortname,pname,pcity,pstate,pzip,hsa,hsaname,...,fcntyname,posbeds,sysname,ahabeds,category,teachstatus,hosptype,nci_center,excl_flag,icu_flag
0,10001,010001,9.OK?,Southeast Alabama Medical Center,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301,1026,AL- DOTHAN,...,AL-Houston County,408,NaN,387.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN
1,10005,010005,9.OK?,Marshall Medical Center South,MARSHALL MEDICAL CENTER SOUTH,BOAZ,AL,35957,1012,AL- BOAZ,...,AL-Marshall County,240,Marshall Health System,114.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN
2,10006,010006,9.OK?,Eliza Coffee Memorial Hospital,ELIZA COFFEE MEMORIAL HOSPITAL,FLORENCE,AL,35631,1036,AL- FLORENCE,...,AL-Lauderdale County,358,RCCH HealthCare Partners,278.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN
3,10007,010007,9.OK?,Mizell Memorial Hospital,MIZELL MEMORIAL HOSPITAL,OPP,AL,36467,1065,AL- OPP,...,AL-Covington County,85,NaN,59.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN
4,10008,010008,9.OK?,Crenshaw Community Hospital,CRENSHAW COMMUNITY HOSPITAL,LUVERNE,AL,36049,1056,AL- LUVERNE,...,AL-Crenshaw County,45,NaN,65.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,noICUdays


In [37]:
hsa_geo2015 = pd.read_csv("~/Downloads/hosp15_5130_atlas.csv")
hsa_geo2015.columns = hsa_geo2015.columns.str.lower()
hsa_geo2015['year'] = 2015
hsa_geo2015.head()

,provider,new_prov,status15,shortname,pcity,pstate,pzip,hsa,hsaname,hrr,...,posbeds,sysname,ahabeds,category,teachstatus,hosptype,nci_center,excl_flag,icu_flag,year
0,10001,NaN,9.OK?,Southeast Alabama Medical Center,DOTHAN,AL,36301,1026,AL- DOTHAN,2,...,408,NaN,420.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2015
1,10005,NaN,9.OK?,Marshall Medical Center South,BOAZ,AL,35957,1012,AL- BOAZ,1,...,240,Marshall Health System,114.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2015
2,10006,NaN,9.OK?,Eliza Coffee Memorial Hospital,FLORENCE,AL,35631,1036,AL- FLORENCE,1,...,358,RCCH HealthCare Partners,278.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2015
3,10007,NaN,9.OK?,Mizell Memorial Hospital,OPP,AL,36467,1065,AL- OPP,2,...,85,NaN,59.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2015
4,10008,NaN,9.OK?,Crenshaw Community Hospital,LUVERNE,AL,36049,1056,AL- LUVERNE,7,...,45,NaN,65.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,noICUdays,2015


In [36]:
hsa_geo2014 = pd.read_csv("~/Downloads/hosp14_5173_atlas.csv",delimiter=';')
hsa_geo2014.columns = hsa_geo2014.columns.str.lower()
hsa_geo2014['year'] = 2014
hsa_geo2014.head()

,provider,new_prov,status14,shortname,pcity,pstate,pzip,hsa,hsaname,hrr,...,posbeds,sysname,ahabeds,category,teachstatus,hosptype,nci_center,excl_flag,icu_flag,year
0,10001,NaN,9.OK?,Southeast Alabama Medical Center,DOTHAN,AL,36301,1026,AL- DOTHAN,2,...,408,NaN,420.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2014
1,10005,NaN,9.OK?,Marshall Medical Center South,BOAZ,AL,35957,1012,AL- BOAZ,1,...,240,Marshall Health System,90.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2014
2,10006,NaN,9.OK?,Eliza Coffee Memorial Hospital,FLORENCE,AL,35631,1036,AL- FLORENCE,1,...,358,RegionalCare Hospital Partners,278.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2014
3,10007,NaN,9.OK?,Mizell Memorial Hospital,OPP,AL,36467,1065,AL- OPP,2,...,87,NaN,59.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2014
4,10008,NaN,9.OK?,Crenshaw Community Hospital,LUVERNE,AL,36049,1056,AL- LUVERNE,7,...,45,NaN,65.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,noICUdays,2014


In [44]:
hsa_geo = pd.concat([hsa_geo2014,hsa_geo2015,hsa_geo2016,hsa_geo2017])
hsa_geo.head()


,provider,new_prov,status14,shortname,pcity,pstate,pzip,hsa,hsaname,hrr,...,teachstatus,hosptype,nci_center,excl_flag,icu_flag,year,status15,status16,pname,status17
0,10001,NaN,9.OK?,Southeast Alabama Medical Center,DOTHAN,AL,36301,1026,AL- DOTHAN,2,...,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2014.0,NaN,NaN,NaN,NaN
1,10005,NaN,9.OK?,Marshall Medical Center South,BOAZ,AL,35957,1012,AL- BOAZ,1,...,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2014.0,NaN,NaN,NaN,NaN
2,10006,NaN,9.OK?,Eliza Coffee Memorial Hospital,FLORENCE,AL,35631,1036,AL- FLORENCE,1,...,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2014.0,NaN,NaN,NaN,NaN
3,10007,NaN,9.OK?,Mizell Memorial Hospital,OPP,AL,36467,1065,AL- OPP,2,...,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,2014.0,NaN,NaN,NaN,NaN
4,10008,NaN,9.OK?,Crenshaw Community Hospital,LUVERNE,AL,36049,1056,AL- LUVERNE,7,...,NaN,4.Other acute care hosp,NaN,9.OK?,noICUdays,2014.0,NaN,NaN,NaN,NaN


In [67]:
hsa_geo.columns

Index(['provider', 'new_prov', 'status14', 'shortname', 'pcity', 'pstate',
       'pzip', 'hsa', 'hsaname', 'hrr', 'hrrname', 'fcounty', 'fcntyname',
       'posbeds', 'sysname', 'ahabeds', 'category', 'teachstatus', 'hosptype',
       'nci_center', 'excl_flag', 'icu_flag', 'year', 'status15', 'status16',
       'pname', 'status17', 'latitude', 'longitude'],
      dtype='object')

In [115]:
mini = hsa_geo[['pzip','pcity']].drop_duplicates(keep='first')


mini[['latitude', 'longitude']] = mini.apply(lambda row: pd.Series(get_coordinates_with_geopy(row)), axis=1)


dothan
31.2237285 -85.3934375
boaz
34.1983281 -86.1649421
florence
34.79981 -87.677251
opp
31.2821209 -86.255839
luverne
31.7165455 -86.2638492
birmingham
33.5206824 -86.8024326
fort payne
34.4442547 -85.7196893
alabaster
33.2442813 -86.8163773
birmingham
33.5206824 -86.8024326
sheffield
34.7650888 -87.6986407
ozark
31.4584764 -85.6405865
centre
34.1520406 -85.6788531
montgomery
32.3669656 -86.3006485
montgomery
32.3669656 -86.3006485
valley
32.8187406 -85.1793852
opelika
32.6454116 -85.3782795
wedowee
33.3090047 -85.484675
tallassee
32.5348568 -85.8922974
cullman
34.1748408 -86.8435802
andalusia
31.3080607 -86.482424
anniston
33.6624824 -85.8283224
huntsville
34.729847 -86.5859011
gadsden
34.0128323 -86.0030251
hamilton
34.1420861 -87.9890206
fayette
33.7303493 -87.7419088
gadsden
34.0128323 -86.0030251
georgiana
31.6371028 -86.7419175
enterprise
31.3151708 -85.8552161
eutaw
32.8406848 -87.8875144
dadeville
32.831238 -85.7635692
dothan
31.2237285 -85.3934375
birmingham
33.5206824 -86.

In [117]:
mini.columns

Index(['pzip', 'pcity', 'latitude', 'longitude'], dtype='object')

In [120]:
hsa_geo.drop(columns=['latitude','longitude'],inplace=True)

In [125]:
final = pd.merge(hsa_geo,mini,on=['pzip','pcity'],how='left')

final.describe()

,provider,pzip,hsa,hrr,fcounty,posbeds,ahabeds,year,latitude,longitude
count,20479.000000,20479.000000,20479.000000,20479.000000,20479.000000,20479.000000,19924.000000,15368.000000,20256.000000,20256.000000
mean,266548.186874,54473.913228,26404.737878,255.908492,29025.488745,147.281899,146.178378,2015.656364,37.803039,-93.575414
std,155757.889820,26303.714552,15046.229902,122.644867,15574.839707,189.940009,188.157198,1.248211,5.390593,14.337981
min,10001.000000,1040.000000,1001.000000,1.000000,1001.000000,1.000000,1.000000,2014.000000,19.202444,-165.424254
25%,140181.500000,33428.000000,14053.000000,170.000000,17069.000000,25.000000,25.000000,2014.000000,33.774629,-98.737591
50%,251336.000000,55805.000000,25089.000000,273.000000,28147.000000,72.000000,70.000000,2016.000000,38.276530,-90.694987
75%,390150.000000,76043.000000,39082.000000,357.000000,42057.000000,199.000000,197.000000,2017.000000,41.503427,-83.113542
max,670124.000000,99929.000000,53026.000000,457.000000,56045.000000,2360.000000,2877.000000,2017.000000,71.290537,-67.840794


In [114]:
def get_coordinates_with_geopy(row):
    city_name = row['pcity'].lower()
    print(city_name)
    zip_code = row['pzip']
    geolocator = Nominatim(user_agent="my_application")
    address = f"{city_name}, {zip_code} United States"
    try:
        location = geolocator.geocode(address)
        if location:
            print(location.latitude, location.longitude)
            return location.latitude, location.longitude
        else:
            return np.nan, np.nan
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return np.nan, np.nan

In [99]:
hsa_geo[['latitude', 'longitude']] = hsa_geo.swifter.apply(lambda row: pd.Series(get_coordinates_with_geopy(row)), axis=1)

dothan
boaz
florence
opp
luverne
birmingham
fort payne
alabaster
birmingham
sheffield
ozark
centre
montgomery
montgomery
valley
opelika
wedowee
birmingham
tallassee
cullman
andalusia
anniston
huntsville
gadsden
hamilton
fayette
gadsden
georgiana
enterprise
eutaw
dadeville
dothan
birmingham
centreville
moulton
scottsboro
geneva
alexander city
eufaula
ashland
anniston
athens
foley
decatur
winfield
mobile
jasper
mobile
grove hill
tuscaloosa
greensboro
wetumpka
brewton
fairhope
talladega
camden
birmingham
birmingham
prattville
carrollton
union springs
demopolis
mobile
bessemer
selma
monroeville
haleyville
troy
jackson
bay minette
pell city
huntsville
york
birmingham
mobile
jacksonville
evergreen
montgomery
greenville
muscle shoals
russellville
sylacauga
phenix city
atmore
chatom
red bay
butler
oneonta
mobile
montgomery
birmingham
birmingham
dothan
anniston
tuscaloosa
alabaster
anchorage
palmer
juneau
fairbanks
anchorage
bethel
soldotna
anchorage
sitka
valdez
seward
sitka
petersburg
wrangel

KeyboardInterrupt: 

In [ ]:
hsa_geo[hsa_geo['latitude'].isna()]

,provider,new_prov,status14,shortname,pcity,pstate,pzip,hsa,hsaname,hrr,...,nci_center,excl_flag,icu_flag,year,status15,status16,pname,status17,latitude,longitude


In [126]:
final.to_csv('../outputs/databases/hsa_geo.csv',index=False)

In [132]:
missing_zips = final[final['latitude'].isna()][['pzip','pcity']].drop_duplicates(keep='first')

In [137]:
for i,row in missing_zips.iterrows():
    print(row['pcity'],row['pzip'],"USA")

HOT SPRINGS 71903 USA
FORT SMITH 72917 USA
GLENDALE 91209 USA
MISSION HILLS 91346 USA
HARTFORD 6105 USA
PUTNAM 6260 USA
SHARON 6069 USA
WATERBURY 6721 USA
BRIDGEPORT 6610 USA
GREENWICH 6830 USA
MILFORD 6460 USA
MIDDLETOWN 6457 USA
NORWICH 6360 USA
HARTFORD 6102 USA
DERBY 6418 USA
DANBURY 6810 USA
NORWALK 6856 USA
FARMINGTON 6032 USA
NEWARK 19718 USA
TAMPA 33677 USA
ALBANY 31703 USA
WA KEENEY 67672 USA
PORTLAND 4101 USA
BRUNSWICK 4011 USA
FALL RIVER 2721 USA
PALMER 1069 USA
BEVERLY 1915 USA
CLINTON 1510 USA
QUINCY 2169 USA
FALL RIVER 2720 USA
W CONCORD 1742 USA
WINCHESTER 1890 USA
LIBERTY 64069 USA
BATTE MTN 89820 USA
CONCORD 3301 USA
LEBANON 3756 USA
DOVER 3820 USA
LANCASTER 3584 USA
LEBANON 3766 USA
JERSEY CITY 7306 USA
JERSEY CITY 7302 USA
LAKEWOOD 8701 USA
WESTWOOD 7675 USA
DOVER 7801 USA
LEXINGTON 27293 USA
CONCORD 44077 USA
MC KEESPORT 15132 USA
PROVIDENCE 2908 USA
NEWPORT 2840 USA
ABERDEEN 57402 USA
BURLINGTON 5401 USA
NEWPORT 5855 USA
SAINT CROIX FALLS 54024 USA
FORT OGLETHORP 3

In [138]:
coords = [(34.465488, -93.070282),
(35.350050, -94.351130),
(34.146320, -118.252020),
(34.281510, -118.462070),
(31.172530, -85.479970),
(42.407610, -79.186660),
(39.047410, -92.240270),
(34.179450, -84.216710),
(43.090970, -76.001470),
(28.218880, -82.700460),
(35.102460, -78.983620),
(39.454890, -85.655030),
(36.800340, -119.903590),
(32.435670, -99.805600),
(43.362580, -82.573310),
(43.561000, -116.267350),
(41.167290, -81.936700),
(43.060940, -88.670070),
(39.690640, -75.671990),
(27.958010, -82.482940),
(31.569220, -84.094660),
(47.527150, -117.414630),
(33.459930, -112.147800),
(42.689850, -88.965110),
(40.399130, -105.570760),
(42.805290, -73.899830),
(35.188480, -80.828090),
(39.661820, -87.476630),
(36.349780, -119.356100),
(40.399060, -105.570820),
(39.556990, -94.469720),
(34.846980, -106.694450),
(39.231180, -94.432790),
(40.635010, -116.933110),
(35.043420, -80.535770),
(38.545410, -89.914910),
(38.772700, -83.960920),
(39.801280, -82.914080),
(34.278180, -83.707660),
(45.586760, -122.744180),
(45.586730, -122.744040),
(35.897830, -78.713500),
(38.118700, -93.597590),
(61.216540, -149.735110),
(35.825640, -80.253680),
(41.712140, -81.239240),
(36.641070, -88.142430),
(41.712550, -88.402890),
(44.002230, -88.604970),
(45.452740, -98.495280),
(27.773980, -82.706920),
(41.942670, -87.773560),
(45.440410, -92.630590),
(34.951980, -85.249060),
(38.249500, -75.295170),
(35.436050, -94.353220),]


In [139]:
missing_zips['latitude'] = [x[0] for x in coords]
missing_zips['longitude'] = [x[1] for x in coords]

In [153]:
for i,row in final.iterrows():
    city = row['pcity']
    zip_code = row['pzip']

    if np.isnan(row['latitude_x']):
        # Filter the missing_zips DataFrame based on the conditions
        matched_rows = missing_zips[(missing_zips['pzip'] == zip_code) & (missing_zips['pcity'] == city)]

        # Check if there is any matching row and then assign values
        if not matched_rows.empty:
            final.at[i, 'latitude_x'] = matched_rows.iloc[0]['latitude']
            final.at[i, 'longitude_x'] = matched_rows.iloc[0]['longitude']
    
   

In [155]:
final.to_csv('../outputs/databases/hsa_geo.csv',index=False)

# Trimming Features 
Coordinating which features should lie in each corresponding table

In [5]:
current_df = '/Users/jeremy.wayland/Desktop/dev/apparent/outputs/databases/iplsd_features.csv'

In [6]:
current_df = pd.read_csv(current_df,index_col=0)

/var/folders/81/4smyjgm56s7gm79ptgb726s904l6lt/T/ipykernel_15506/2679802802.py:1: DtypeWarning: Columns (318) have mixed types. Specify dtype option on import or set low_memory=False.
  current_df = pd.read_csv(current_df,index_col=0)


In [7]:
for col in current_df.columns:
    print(col)

hsanum
region
state
medicare_enrollment
total_mortality
non_hmo_mortality
forman_curvature
n_nodes
n_edges
density
assortativity
average_degrees
clustering_coefficients
node_connectivities
degree_correlations
obd_overall_diab_bloodlip
obd_nonblk_diab_bloodlip
obd_blk_diab_bloodlip
obd_blk_diab_eye
obd_overall_diab_eye
obd_nonblk_diab_eye
obd_overall_diab_hgb
obd_blk_diab_hgb
obd_nonblk_diab_hgb
obd_overall_mam_mam6769
obd_nonblk_mam_mam6769
obd_blk_mam_mam6769
obd_nonblk_ptbjune_amcare2
obd_blk_ptbjune_amcare2
obd_overall_ptbjune_amcare2
oe_overall_diab_bloodlip
oe_nonblk_diab_bloodlip
oe_blk_diab_bloodlip
oe_blk_diab_eye
oe_overall_diab_eye
oe_nonblk_diab_eye
oe_overall_diab_hgb
oe_blk_diab_hgb
oe_nonblk_diab_hgb
oe_overall_mam_mam6769
oe_nonblk_mam_mam6769
oe_blk_mam_mam6769
oe_nonblk_ptbjune_amcare2
oe_blk_ptbjune_amcare2
oe_overall_ptbjune_amcare2
cr_overall_diab_bloodlip
cr_nonblk_diab_bloodlip
cr_blk_diab_bloodlip
cr_blk_diab_eye
cr_overall_diab_eye
cr_nonblk_diab_eye
cr_overall_

In [8]:
current_df.columns = current_df.columns.str.lower()

In [15]:
current_df.forman_curvature = current_df.forman_curvature.apply(lambda x: np.array(x).astype(float))

ValueError: could not convert string to float: '[-2.0, 0.0, -2.0, -3.0, 0.0, 3.0, 1.0]'

In [11]:
def calculate_statistics(df, column):
    """
    Calculates mean, standard deviation, mode, median, and quantiles for a column of lists in a DataFrame.
    
    Args:
    df (pd.DataFrame): The DataFrame containing the column with lists.
    column (str): The name of the column with lists of curvature values.
    
    Returns:
    pd.DataFrame: The original DataFrame with new columns for each statistic.
    """
    # Helper function to compute statistics
    def compute_stats(values):
        mean_val = np.mean(values)
        stdev_val = np.std(values)
        mode_val = stats.mode(values)[0][0]
        median_val = np.median(values)
        quantiles = np.percentile(values, [25, 50, 75])
        return pd.Series({
            'mean': mean_val,
            'stdev': stdev_val,
            'mode': mode_val,
            'median': median_val,
            'quantile_25': quantiles[0],
            'quantile_50': quantiles[1],  # same as median
            'quantile_75': quantiles[2]
        })
    
    # Apply the helper function to each row in the specified column
    stats_df = df[column].apply(compute_stats)
    
    # Concatenate the original DataFrame with the new statistics DataFrame
    df = pd.concat([df, stats_df], axis=1)
    
    return df

In [12]:
calculate_statistics(current_df, 'forman_curvature')

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U38'), dtype('<U38')) -> None

In [6]:


df = pd.read_csv('../outputs/IPLDS_all_network_features.csv')

In [15]:
df2 = pd.read_csv('../outputs/databases/iplsd_features.csv',index_col=0).drop(columns=["forman_mean","density"])

/var/folders/81/4smyjgm56s7gm79ptgb726s904l6lt/T/ipykernel_19570/75855171.py:1: DtypeWarning: Columns (318) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('../outputs/databases/iplsd_features.csv',index_col=0).drop(columns=["forman_mean","density"])


In [12]:
pd.merge(df, df2, left_on='hsa', right_on='HSA',)

,hsa,year,assortativity_x,average_clustering,density_x,forman_mean_x,forman_stdev,forman_median,forman_quantile_25,forman_quantile_50,...,SYSNAME,AHAbeds,CATEGORY,TEACHSTATUS,HOSPTYPE,NCI_CENTER,EXCL_FLAG,ICU_FLAG,latitude,longitude
0,13030,2016,-0.107466,0.446798,0.140592,-2.330827,6.694989,-2.0,-6.0,-2.0,...,NaN,69.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,43.8100,-111.7890
1,4002,2017,-0.590909,0.300000,0.333333,-0.428571,1.916630,0.0,-2.0,0.0,...,NaN,25.0,NaN,NaN,3.Critical Access Hosp,NaN,9.OK?,noICUdays,33.6787,-94.1351
2,14026,2014,-0.641791,0.542857,0.476190,1.100000,1.920937,2.0,0.0,2.0,...,NaN,25.0,NaN,NaN,3.Critical Access Hosp,NaN,9.OK?,NaN,40.1487,-88.9627
3,17037,2017,-0.234387,0.808016,0.290472,-1.968153,12.181442,1.0,-11.0,1.0,...,University of Kansas Health System,33.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,38.3936,-98.7751
4,17037,2017,-0.234387,0.808016,0.290472,-1.968153,12.181442,1.0,-11.0,1.0,...,University of Kansas Health System,22.0,NaN,NaN,3.Critical Access Hosp,NaN,9.OK?,noICUdays,38.1946,-99.1010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12035,23042,2017,0.061726,0.592018,0.075720,-8.722184,20.632640,-6.0,-22.0,-6.0,...,Beaumont Health,280.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,42.3847,-82.9244
12036,23042,2017,0.061726,0.592018,0.075720,-8.722184,20.632640,-6.0,-22.0,-6.0,...,Ascension Healthcare,621.0,Independent,Major Teaching,2.Other teaching hosp,NaN,9.OK?,NaN,42.4274,-82.9002
12037,23042,2017,0.061726,0.592018,0.075720,-8.722184,20.632640,-6.0,-22.0,-6.0,...,NaN,NaN,NaN,NaN,5.Longterm/non-acute,NaN,1.NonAcute,noICUdays,42.4274,-82.9002
12038,5136,2016,-0.426011,0.328911,0.038924,-23.024390,20.095893,-22.0,-45.0,-22.0,...,NaN,113.0,NaN,NaN,4.Other acute care hosp,NaN,9.OK?,NaN,37.9864,-122.1350


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/jeremy.wayland/Desktop/dev/apparent/outputs/databases/network_panel_undirected_local_hsa_edges.csv')

In [4]:
len(df)/4

9785175.0